<a href="https://colab.research.google.com/github/Psyclophe/Data-Scientist/blob/main/Wal_mart_analysis_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
from pandas.core.indexes.datetimes import date
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels as sm
import sklearn as sl

!pip install pmdarima

import pmdarima as pm
from pmdarima.arima import auto_arima
from pmdarima.model_selection import train_test_split
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import BoxCoxEndogTransformer
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt

import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp as multi

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp as mutti

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.compat.scipy import _next_regular

import warnings
warnings.filterwarnings ("ignore")

from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## sección 1.***Obtención de Datos***
es este apartado se sube la información del dataFrame en github





In [2]:
#se cargan los datos de entrada proporcionado en el proyecto de las tiendas Wal-mart, asimismo se hace la conexión de los datos a Github mostrado en la siguiente ruta
#de la misma forma se indica a Python muestre los primeros "15" registros

url = 'https://raw.githubusercontent.com/Psyclophe/Data-Scientist/main/Walmart_Store_sales.csv'
df = pd.read_csv (url)
print('Shape of data',df.shape)
df.head(15)

Shape of data (6435, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106
5,1,12-03-2010,1439541.59,0,57.79,2.667,211.380643,8.106
6,1,19-03-2010,1472515.79,0,54.58,2.720,211.215635,8.106
7,1,26-03-2010,1404429.92,0,51.45,2.732,211.018042,8.106
8,1,02-04-2010,1594968.28,0,62.27,2.719,210.820450,7.808
9,1,09-04-2010,1545418.53,0,65.86,2.770,210.622857,7.808


## sección 2. **Análisis Exploratorio**
en esta sección se procede a realizar el análisis de la información proporcionada con Python

In [3]:
#para identificar los datos nulos se indica a Python que nos muestre los datos y, poder detectar la cantidad de datos nulos

df.isnull().sum()

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [4]:
#con la carga de informacion y con esta funcion se pide a Python indicar el tipo de dato de cada columna, 
#para analizar si es necesario hacer cambios en el tipo de dato en el dataframe y poder tener datos homogeneos
#se identifico que "Date" es un dato que es objeto y se debe convertir a dato fecha

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [5]:
df.dtypes

Store             int64
Date             object
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [6]:
#en esta funcion se pide a Python convertir los datos objeto a string
#aquí podemos visualizar que el dato que era un objeto "Date" se conviritio al formato fecha en donde se visualiza "datetime64[ns]"

df["Date"]=pd.to_datetime(df["Date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         6435 non-null   int64         
 1   Date          6435 non-null   datetime64[ns]
 2   Weekly_Sales  6435 non-null   float64       
 3   Holiday_Flag  6435 non-null   int64         
 4   Temperature   6435 non-null   float64       
 5   Fuel_Price    6435 non-null   float64       
 6   CPI           6435 non-null   float64       
 7   Unemployment  6435 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 402.3 KB


In [7]:
#con este script se pide a Pyhon poner en formato de dos digitos y dividir por comas las cifras de la informacion de datframe

pd.options.display.float_format='{:,.2f}'.format

In [8]:
#con esta funcion se da la instrucción a Python para ver las medidas de tendencia central de las 8 columnas

df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,"6,435.00","6,435.00","6,435.00","6,435.00","6,435.00","6,435.00","6,435.00"
mean,23.00,"1,046,964.88",0.07,60.66,3.36,171.58,8.00
std,12.99,"564,366.62",0.26,18.44,0.46,39.36,1.88
min,1.00,"209,986.25",0.00,-2.06,2.47,126.06,3.88
25%,12.00,"553,350.10",0.00,47.46,2.93,131.74,6.89
50%,23.00,"960,746.04",0.00,62.67,3.44,182.62,7.87
75%,34.00,"1,420,158.66",0.00,74.94,3.73,212.74,8.62
max,45.00,"3,818,686.45",1.00,100.14,4.47,227.23,14.31


In [9]:
#se da la instruccion a Python que se ordene las fechas de menor a mayor en el archivo que se va a analizar
#asimismo se indica al programa que ordene por fechas, de la mas antigua hasta la mas reciente

data_sort=df.sort_values("Date")
data_sort

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
606,5,2010-01-10,"283,178.12",0,71.10,2.60,212.23,6.77
2036,15,2010-01-10,"566,945.95",0,59.69,2.84,132.76,8.07
5897,42,2010-01-10,"481,523.93",0,86.01,3.00,126.23,9.00
4610,33,2010-01-10,"224,294.39",0,91.45,3.00,126.23,9.27
5039,36,2010-01-10,"422,169.47",0,74.66,2.57,210.44,8.48
...,...,...,...,...,...,...,...,...
5860,41,2012-12-10,"1,409,544.97",0,39.38,3.76,199.05,6.20
2285,16,2012-12-10,"491,817.19",0,43.26,3.76,199.05,5.85
1427,10,2012-12-10,"1,713,889.11",0,76.03,4.47,131.11,6.94
3572,25,2012-12-10,"697,317.41",0,43.74,4.00,216.12,7.29


In [10]:
#con la funcion "groupby" se indica a Python agrupar y sumar, por tienda y por semana, al ejecutar la funcion Python
#agrega e indexa la columna "0" a la columna "1" como identificador

group=df.groupby("Store").sum("Weekly_Sales")
group.reset_index(level=0, inplace=True)
group["Weekly_Sales"]=(group["Weekly_Sales"])
group

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,"222,402,808.85",10,"9,767.87",460.42,"30,887.56","1,088.29"
1,2,"275,382,440.98",10,"9,754.94",460.42,"30,837.42","1,090.21"
2,3,"57,586,735.07",10,"10,215.09",460.42,"31,372.99","1,026.31"
3,4,"299,543,953.38",10,"8,902.23",460.03,"18,401.19",852.95
4,5,"45,475,688.90",10,"9,925.65",460.42,"30,968.88",900.24
5,6,"223,756,130.64",10,"9,967.10",460.42,"31,110.11",944.79
6,7,"81,598,275.14",10,"5,680.00",463.54,"27,693.99","1,227.76"
7,8,"129,951,181.13",10,"8,939.50",460.42,"31,379.78",871.13
8,9,"77,789,218.99",10,"9,691.85",460.42,"31,406.62",872.28
9,10,"271,617,713.89",10,"10,330.49",511.36,"18,401.19","1,195.90"


In [11]:
#con este script se pide a Python separar el dato "Date" a year, quarter, month, week & day, datos que se utiizaran

df["Year"]= df['Date'].dt.year
df["Quarter"]= df['Date'].dt.quarter
df["Month"]= df['Date'].dt.month
df["Week"]= df['Date'].dt.week
df["Day"]= df['Date'].dt.day
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Quarter,Month,Week,Day
0,1,2010-05-02,"1,643,690.90",0,42.31,2.57,211.10,8.11,2010,2,5,17,2
1,1,2010-12-02,"1,641,957.44",1,38.51,2.55,211.24,8.11,2010,4,12,48,2
2,1,2010-02-19,"1,611,968.17",0,39.93,2.51,211.29,8.11,2010,1,2,7,19
3,1,2010-02-26,"1,409,727.59",0,46.63,2.56,211.32,8.11,2010,1,2,8,26
4,1,2010-05-03,"1,554,806.68",0,46.50,2.62,211.35,8.11,2010,2,5,18,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,"713,173.95",0,64.88,4.00,192.01,8.68,2012,3,9,39,28
6431,45,2012-05-10,"733,455.07",0,64.89,3.98,192.17,8.67,2012,2,5,19,10
6432,45,2012-12-10,"734,464.36",0,54.47,4.00,192.33,8.67,2012,4,12,50,10
6433,45,2012-10-19,"718,125.53",0,56.47,3.97,192.33,8.67,2012,4,10,42,19


In [12]:
df.set_index('Quarter')

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Day
Quarter,,,,,,,,,,,,
2,1,2010-05-02,"1,643,690.90",0,42.31,2.57,211.10,8.11,2010,5,17,2
4,1,2010-12-02,"1,641,957.44",1,38.51,2.55,211.24,8.11,2010,12,48,2
1,1,2010-02-19,"1,611,968.17",0,39.93,2.51,211.29,8.11,2010,2,7,19
1,1,2010-02-26,"1,409,727.59",0,46.63,2.56,211.32,8.11,2010,2,8,26
2,1,2010-05-03,"1,554,806.68",0,46.50,2.62,211.35,8.11,2010,5,18,3
...,...,...,...,...,...,...,...,...,...,...,...,...
3,45,2012-09-28,"713,173.95",0,64.88,4.00,192.01,8.68,2012,9,39,28
2,45,2012-05-10,"733,455.07",0,64.89,3.98,192.17,8.67,2012,5,19,10
4,45,2012-12-10,"734,464.36",0,54.47,4.00,192.33,8.67,2012,12,50,10


In [13]:
print(df.columns)
print(df.shape)
print(df.head(5))
print(df.index.name)

Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Quarter', 'Month', 'Week',
       'Day'],
      dtype='object')
(6435, 13)
   Store       Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1 2010-05-02  1,643,690.90             0        42.31        2.57   
1      1 2010-12-02  1,641,957.44             1        38.51        2.55   
2      1 2010-02-19  1,611,968.17             0        39.93        2.51   
3      1 2010-02-26  1,409,727.59             0        46.63        2.56   
4      1 2010-05-03  1,554,806.68             0        46.50        2.62   

     CPI  Unemployment  Year  Quarter  Month  Week  Day  
0 211.10          8.11  2010        2      5    17    2  
1 211.24          8.11  2010        4     12    48    2  
2 211.29          8.11  2010        1      2     7   19  
3 211.32          8.11  2010        1      2     8   26  
4 211.35          8.11  2010        2      5    18    

In [14]:
dfd = df.iloc[:,1:13]
print(dfd.columns)

Index(['Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price',
       'CPI', 'Unemployment', 'Year', 'Quarter', 'Month', 'Week', 'Day'],
      dtype='object')


In [33]:
dfp=df.iloc[:,1:13]
dfp.columns = pd.to_datetime(dfp.columns, dayfirst=True, errors='ignore')
print(dfp.head(5))

        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price    CPI  \
0 2010-05-02  1,643,690.90             0        42.31        2.57 211.10   
1 2010-12-02  1,641,957.44             1        38.51        2.55 211.24   
2 2010-02-19  1,611,968.17             0        39.93        2.51 211.29   
3 2010-02-26  1,409,727.59             0        46.63        2.56 211.32   
4 2010-05-03  1,554,806.68             0        46.50        2.62 211.35   

   Unemployment  Year  Quarter  Month  Week  Day  
0          8.11  2010        2      5    17    2  
1          8.11  2010        4     12    48    2  
2          8.11  2010        1      2     7   19  
3          8.11  2010        1      2     8   26  
4          8.11  2010        2      5    18    3  


In [35]:
#dfp = dfp.groupby(pd.PeriodIndex(dfp.columns, freq='M'), axis =1).sum()
#print(dfp.head(5))



In [39]:
sum_monthly=dfp.sum(axis=0)
print(sum_monthly)

Weekly_Sales   6,737,218,987.11
Holiday_Flag             450.00
Temperature          390,371.44
Fuel_Price            21,612.64
CPI                1,104,106.96
Unemployment          51,474.54
Year              12,940,560.00
Quarter               16,065.00
Month                 41,670.00
Week                 167,310.00
Day                  100,710.00
dtype: float64


In [ ]:
#print(df.groupby(['Weekly_Sales', 'Quarter']).sum())

In [ ]:
#con este script se pide a Python agrupar y sumar por Quarter

#df=[['Date']]
#df=[['Quarter']]
#df.iloc[0:6,8:11]
#df[['Quarter']]


In [ ]:
#df=pd('Quarter')

#df_Quarter=df["Date"].dt.quarter
#Period=df[period.third_quarter('Date')]
#df = df.index.quarter
#group=df.groupby(df.index.quarter).head(1).reset_index()
#group


In [ ]:
#se solicita a Python ordenar por tienda y por volumen de ventas, del valor mayor al menor

sort_WeeklySales=group.sort_values('Weekly_Sales', ascending=False)
sort_WeeklySales

In [ ]:
#con este script le indicamos a Python que solo nos indique el Top_five de las ventas por tienda

Top_five=sort_WeeklySales[['Weekly_Sales', 'Store']][0:5:].reset_index(drop=True)
Top_five

In [ ]:
#es este script se solicita a Python calcular la Desviacion Estandar por tienda, ordenanda de mayor a menor

Store_devstd = df.groupby('Store')['Weekly_Sales'].std()
Store_devstd = pd.DataFrame(Store_devstd)
Store_devstd=Store_devstd.sort_values('Weekly_Sales', ascending=False)
Store_devstd

In [ ]:
#con este script Python hace el calculo de la tienda con más desviacion estandar

top_std = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std().sort_values(ascending=False))
top_std.head(1).index[0] , top_std.head(1).Weekly_Sales[top_std.head(1).index[0]] 

In [ ]:
# con este script se determina el -Coeficiente de Desviación Medio- por tienda el
#se determino dividiendo la DesvStd por tienda entre la media de las tiendas, como sigue coeficiente_DM = Desviación Estandar / media aritmetica * 100

coeficiente_DM = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std() / df.groupby('Store')['Weekly_Sales'].mean()*100)
coeficiente_DM

In [ ]:
#con este script se pide a Python identificar los días festivos

df_Holiday=df[df["Holiday_Flag"]==1]
df_Holiday

In [ ]:
#con esta instruccion se definen los Días festivos que encontro Pyton en el dataframe

Super_Bowl =['2010-12-02', '2011-11-02', '2012-10-02']
Labor_Day =  ['2010-10-09', '2011-09-09', '2012-07-09']
Thanksgivig_Day =  ['2010-11-26', '2011-11-25', '2012-11-23']
Xmas = ['2010-12-31', '2011-12-30', '2012-12-28']

In [ ]:
#con este script se pide a Python sumar los dias festivos

add_days = Super_Bowl + Labor_Day + Thanksgivig_Day + Xmas
add_days
df["Holiday_Flag"] = np.where(df["Holiday_Flag"] == 1, df["Holiday_Flag"], df["Holiday_Flag"].isin(add_days))
df['Holiday_Flag'].value_counts()

In [ ]:
from scipy import stats
x=['Holiday_Flag']
y=['Weekly_Sales']
stats.kruskal(x,y)

In [ ]:
x=df[df['Holiday_Flag'] == 0]['Weekly_Sales']
y=df[df['Holiday_Flag'] == 1]['Weekly_Sales']
stats.kruskal(x,y)

In [ ]:
df.loc[df.Date.isin(Super_Bowl)].Date.unique()

In [ ]:
df.loc[df.Date.isin(Labor_Day)].Date.unique()

In [ ]:
df.loc[df.Date.isin(Thanksgivig_Day)].Date.unique()

In [ ]:
df.loc[df.Date.isin(Xmas)].Date.unique()

In [ ]:
#con esta instruccion se pide a Python determinar la media de ventas de los Dias_Festivos

Sales_Super_Bowl= (pd.DataFrame(df.loc[df.Date.isin(Super_Bowl)]))['Weekly_Sales'].mean()
Sales_Labor_Day = (pd.DataFrame(df.loc[df.Date.isin(Labor_Day)]))['Weekly_Sales'].mean()
Sales_Thanksgivig_Day = (pd.DataFrame(df.loc[df.Date.isin(Thanksgivig_Day)]))['Weekly_Sales'].mean()
Sales_Xmas = (pd.DataFrame(df.loc[df.Date.isin(Xmas)]))['Weekly_Sales'].mean()
{Sales_Super_Bowl, Sales_Labor_Day, Sales_Thanksgivig_Day, Sales_Xmas}

In [ ]:
#con esta instruccion se pide a Python determinar la media de ventas de los Dias_No_Festivos

Sales_not_Holiday = df[df['Holiday_Flag'] == 0 ]['Weekly_Sales'].mean()
{Sales_not_Holiday}

In [ ]:
#con esta instruccion se pide a Python determinar el promedio de ventas 

Average_sales = {'Sales_Super_Bowl' : Sales_Super_Bowl,
                 'Sales_Labor_Day': Sales_Labor_Day,
                 'Sales_Thanksgiving_Day':Sales_Thanksgivig_Day,
                 'Sales_Xmas': Sales_Xmas,
                 'Sales_not_Holiday': Sales_not_Holiday}
Average_sales 

In [ ]:
Sales_Quarter=df.groupby('Quarter')['Weekly_Sales'].sum()
Sales_Quarter

In [ ]:
df.groupby('Quarter').sum()

In [ ]:
from pandas.core.groupby.groupby import DataFrame
#con este script se pide a Python realizar la suma por Quarter

pd=DataFrame({'Year', 'Quarter'})
print(df.groupby(['Year', 'Quarter']).sum().groupby(level=[0]).cumsum())

In [ ]:
#con este script se pide a Python sumar el total de ventas por Quarter
Sales_Year=df.groupby('Year')['Weekly_Sales'].sum()
Sales_Year

In [ ]:
#df = df.set_index['Date']
#group=df.groupby(df.index.quarter).head(1).reset_index()
#print('Shape of data',df.shape)
#group

In [ ]:
#con este script se muestra en crecimiento tercer trimestre 2012 por tienda

#third_quarter= df[(df['Date'] >= '2012-07-01') & (df['Date'] <= '2012-09-30')].groupby('Store')['Weekly_Sales'].sum().sort_values()
#plt.figure(figsize=(15,7))
#third_quarter.plot(kind='bar',legend=True)
#plt.ticklabel_format( style='plain', axis='y')
#plt.legend(["3-Quarter' 2012"]);


In [ ]:
#con este script se pide a Python graficar las Ventas mensuales por cada año 

plt.figure(figsize=(20,8))
plt.scatter(df[df.Year==2010]["Month"],df[df.Year==2010]["Weekly_Sales"])
plt.xlabel("Months")
plt.ylabel("Weekly Sales")
plt.title("Monthly view of sales in 2010")
plt.show()

plt.figure(figsize=(20,8))
plt.scatter(df[df.Year==2011]["Month"],df[df.Year==2011]["Weekly_Sales"])
plt.xlabel("Months")
plt.ylabel("Weekly Sales")
plt.title("Monthly view of sales in 2011")
plt.show()

plt.figure(figsize=(20,8))
plt.scatter(df[df.Year==2012]["Month"],df[df.Year==2012]["Weekly_Sales"])
plt.xlabel("Months")
plt.ylabel("Weekly Sales")
plt.title("Monthly view of sales in 2012")
plt.show()

In [ ]:
#con esta instrucción se pide a Python graficar las ventas de los tres años de manera mensual

#plt.figure(figsize=(20,8))
#sns.lineplot(data=df, x='Month', y='Weekly_Sales', hue='Year', palette='tab10')
#sns.set_style('whitegrid')
#sns.despine(left=True, offset=20, trim=True)

#plt.show()

# sección 3. **Quitar Valores de Rango**
en esta sección del codigo se analiza la información para quitar los valores fuera de rango


In [ ]:
#con este script se grafican y validan los valores atipicos

fig, axis = plt.subplots(4,figsize=(16,16))
X = df[['Temperature','Fuel_Price','CPI','Unemployment']]
for i,column in enumerate(X):
    sns.boxplot(df[column],ax=axis[i])

In [ ]:
sns.scatterplot(data=df,x='Temperature', y='Weekly_Sales', hue='Year', style='Year')
#sns.scatterplot(data=df,x='Temperature', y='Weekly_Sales', hue='Year')
plt.axvline(x=10,color='b')

In [ ]:

#sns.scatterplot(data=df,x='Holiday_Flag', y='Temperature')


In [ ]:
sns.scatterplot(data=df,x='Unemployment', y='Weekly_Sales', hue='Year', style='Year')
plt.axvline(x=4.5,color='b')

In [ ]:
#con este script pedimos aPython eliminar los valores atipicos en una nueva función

df_clean = df[(df['Unemployment']<10) & (df['Unemployment']>4.5) & (df['Temperature']>10)]


In [ ]:
#lim_inf = Q1 - 1.5(whis) IQR (scipy.stats.iqr)
#iqr = Q3 - Q1
#lim_sup = Q3 + 1.5(whis) IQR

In [ ]:
# Verificamos ya no tener valores atipicos

fig, axis = plt.subplots(4,figsize=(16,16))
X = df_clean[['Temperature','Fuel_Price','CPI','Unemployment']]
for i,column in enumerate(X):
    sns.boxplot(df_clean[column],ax=axis[i])


In [ ]:
#con este grafico de calor podemos ver las correlaciones de las diferentes columnas

plt.figure(figsize = (12,12))
sns.heatmap(df_clean.corr().abs())    

plt.show()

In [ ]:
#con esta instrucción se pide a Python graficar las ventas de los tres años por Quarter

#plt.figure(figsize=(20,8))
#sns.lineplot(data=df, x='Month', y='Weekly_Sales', hue='Quarter', palette='tab10')
#sns.set_style('whitegrid')
#sns.despine(left=True, offset=20, trim=True)

#plt.show()

# sección 4. **Modelos y Pronostico**

en esta sección del codigo se procede a realizar el modelado de la información

In [ ]:
#sns.distplot=sns.distplot('Quarter')

In [ ]:
import prophet as ppt

In [ ]:
#con este script se solicita a Python calcular el analsis de variaza entre 'Holiday_ Flag' vs 'Weekly_Sales', el modelo indica que la variacion
#es pequeña indicando al modelo que los Holiday no afecta las ventas semanales de manera significativa

model = smf.ols ('Holiday_Flag ~ Weekly_Sales', data = df).fit()
aov_table = anova_lm(model, type=2)

#anova_result = sm.stats.anova_lm(model, type=2)
print(aov_table)

In [ ]:
import scipy.stats as stats
from pmdarima import auto_arima

In [ ]:
#df_arima=df
#df_arima=df_arima.set_index('Date').reset.index()

In [ ]:
#plt.figure(figsize=(20,9))
#df_arima['Weekly_Sales'].plot()
#plt.show()

In [ ]:
#df_arimamonth = df_arima.resample('M').mean()

In [ ]:
#plt.figure(figsize=(20,8))
#df_arimamonth['Weekly_Sales'].plot()
#plt.title('Average Sales - Weekly')
#plt.show()

In [ ]:
#train_data = df_arimamonth[:int(0.7*(len(df_arimamonth)))] 
#test_data = df_arimamonth[int(0.7*(len(df_arimamonth))):]

#print('Train:', train_data.shape)
#print('Test:', test_data.shape)

In [ ]:
train_data['Weekly_Sales'].plot(figsize=(20,8), title= 'Weekly_Sales', fontsize=14)
test_data['Weekly_Sales'].plot(figsize=(20,8), title= 'Weekly_Sales', fontsize=14)
plt.show()

In [ ]:
from pmdarima.arima import ADFTest
adf_test = ADFTest(alpha = 0.05)
adf_test.should_diff (df_arimamonth['Weekly_Sales'])

In [ ]:
model_auto_arima = auto_arima(train_data['Weekly_Sales'], trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
                  max_p=20, max_q=20, max_P=20, max_Q=20, seasonal=True,maxiter=200,
                  information_criterion='aic',stepwise=False, suppress_warnings=True, D=0, max_D=0,
                  error_action='ignore',approximation = False)
model_auto_arima.fit(train_data['Weekly_Sales'])

In [ ]:
model_auto_arima.summary()

In [ ]:
y_pred = model_auto_arima.predict(n_periods=len(test_data['Weekly_Sales']))
y_pred = pd.DataFrame(y_pred,index = test_data['Weekly_Sales'].index,columns=['Prediction'])
plt.figure(figsize=(20,6))
plt.title('Prediction of Weekly Sales Using Auto-ARIMA', fontsize=20)
plt.plot(train_data['Weekly_Sales'], label='Train')
plt.plot(test_data['Weekly_Sales'], label='Test')
plt.plot(y_pred, label='Prediction of ARIMA')
plt.legend(loc='best')
plt.xlabel('Date', fontsize=14)
plt.ylabel('Weekly Sales', fontsize=14)
plt.show()

In [ ]:
#con este escript se elige el tipo de modelo

#import statsmodels as sm
#from statsmodels.tsa.arima_model import ARIMA
#from statsmodels.tsa.stattools import adfuller
#def test_stationarity(timeseries):

#estadisticas en la determinación de la media móvil y la desviación estandar
#  rolmean = pd.rolling_mean(timeseries, window=12)
#  rolstd = pd.rolling_std(timeseries, window=12)

#graficar las estadisticas
#  orig = plt.plot(timeseries, color="blue", label="Original")
#  mean = plt.plot(rolmean, color="red", label="Media Movil")
#  std = plt.plot(rolstd, color="black", label="Desviacon Estandar")
#  plt.legend(loc="best")
#  plt.title("Media Movil y Desviacion Estandar")
#  plt.show(block=False)


#calculo de la media movil
#moving_avg = pd.rolling_mean(datos["Weekly_Sales"], 12)
#datos["Weekly_Sales"]

#grafica de la media movil
#moving_avg.plot(label="Media Movil", color="red")
#plt.legend(loc="best")



#from statsmodels.tsa.stattools import adfuller
#def test_stationarity(timeseries):
    #determine rolling statistics
#    rolmean = pd.Series(timeseries).rolling(window=24).mean()#24 hours on each day
#    rolstd = pd.Series(timeseries).rolling(window=24).std()
#    #plot rolling statistics
#    orig = plt.plot(timeseries,color = 'blue',label='original')
#    mean = plt.plot(rolmean,color = 'red',label = 'rolling mean')
#    std = plt.plot(rolstd,color = 'black',label = 'rolling std')
#    plt.legend(loc = 'best')
#    plt.title('rolling mean and standard deviation')
#    plt.show(block = False)

   #perform dickey fuller test
#    print('result of dickey fuller test:')
#    dftest = adfuller(timeseries,autolag = 'AIC')
#    dfoutput = pd.Series(dftest[0:4],index = ['Test statistics', 'p-value', '#lags used', 'number of observation used'])
#    for key,value in dftest[4].items():
#        dfoutput['critical value (%s)'%key] = value
#    print(dfoutput)

#from matplotlib.pylab import rcParams
#rcParams['figure.figsize'] = 20,10
#test_stationarity(train_original['Count'])


In [ ]:
#df = px.data.gapminder()
#Weekly_Sales = df.query("Weekly_Sales")

#for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
#    fig = px.scatter(Weekly_Sales,
#                     x="gdpPercap", y="lifeExp", size="pop", color="continent",
#                     log_x=True, size_max=60,
#                     template=template, title="Gapminder 2007: '%s' theme" % template)
#    fig.show()


#df = px.data.gapminder()
#df_2007 = df.query("year==2007")

#for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
#    fig = px.scatter(df_2007,
#                     x="gdpPercap", y="lifeExp", size="pop", color="continent",
#                     log_x=True, size_max=60,
#                     template=template, title="Gapminder 2007: '%s' theme" % template)
#    fig.show()

In [ ]:
#f.["Temperature"].plot(figsize=(20,8))

#df.plot()

In [ ]:
#plt.plot(test)

train = df_clean['Weekly_Sales']
test = df_clean['Weekly_Sales']
plt.plot(train)
plt.plot(test)


In [ ]:
plt.figure(figsize=(50,6))
df_clean['Weekly_Sales'].plot()

In [ ]:
#model_auto_arima = auto_arima(train_data_diff, trace=True,start_p=0, start_q=0, start_P=0, start_Q=0,
#                  max_p=20, max_q=20, max_P=20, max_Q=20, seasonal=True,maxiter=200,
#                  information_criterion='aic',stepwise=False, suppress_warnings=True, D=1, max_D=10,
#                  error_action='ignore',approximation = False)
#model_auto_arima.fit(train_data_diff)

In [ ]:
X = df_clean[['Store','Fuel_Price','CPI','Unemployment','Day','Month','Year']]
y = df_clean['Weekly_Sales']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
model.coef_

In [ ]:
X.mean()